# 8 - 2D Prepopulation Construction:

### In this tutorial we will learn how to prepopulate constructions in 2D for down-selecting 3D generation.
### This will involve 3 key takeaways:

**(A)** How to generate in 2D.

**(B)** How to translate from 2D to 3D in Architector.

**(C)** How to perform 2D to 3D generation in an end-to-end workflow.

In [ ]:
# First, imports:
from architector import (view_structures,
                         build_complex,
                         build_complex_2D) # 2D construction routine!
import pandas as pd # library for handling tables (think Excel charts!)
import copy # Use the copy library

# Now, let's come up with a toy problem.

### (A) Here, let's prepopulate a set of 2D structures for all of the lanthanides with coordination number 5-10 surrounded by waters!

It will be much easier to do this in 2D first - than pick ones we want to do in 3D.

In [ ]:
# First we will build a container input dictionary 
inputDict = {'core':{'metal':'La','coreCN':5}, # Fill the dictionary with 5
             'ligands':['water'],
             'parameters':{'fill_ligand':0} # Fill out the coordination environment with water!
            }

Next, we will use this example simple for loops to generate in 2D.

In [ ]:
# This should take just a fraction of second!
out = build_complex_2D(inputDict)

Let's see what's in this 2D output dictionary:

In [ ]:
out

Looks like just a mol2string, and an output dictionary giving the same as the input.
Let's look at the mol2string:

In [ ]:
print(out['mol2string'])

Notice the structure contains the correct bonds, along with the Charge, and Unpaired Electrons in the system in the header, but no X-Y-Z coordinates (3D information!)

Let's change that.

# (B) Now, let's translate this 2D mol2string into 3D.

To do this let's prepopulate a dictionary - note that all you need is the mol2string from 2D.
Architector will handle the translation internally!

In [ ]:
translate_dict = {'mol2string':out['mol2string'], 
                  'parameters':{}}

Onto 3D generation!

In [ ]:
out_3D = build_complex(translate_dict)

The out_3D dictionary should behave just like other architector output dictionaries, and we now have 3D mol2strings to visualize!

In [ ]:
view_structures(out_3D)

Looks great!

### (C) Now, we can prepopulate all the structures we want to generate in 2D.

Here, we will just use a two nested for loops. This block is a bit larger to account for generating it all in one go! But should just take a couple seconds.

In [ ]:
import architector.io_ptable as io_ptable # Import the periodic table from Architector

metals = [] # Get empty lists ready for these parameters!
coordination_numbers = []
mol2strings = []

for metal in io_ptable.lanthanides: # Iterate over the lanthanide elements
    for cn in range(5,11): # Iterate over all desired coordinations
        metals.append(metal) # Save the metal
        coordination_numbers.append(cn) # Save the cn
        inpDict = copy.deepcopy(inputDict) # Copy from our previous 2D dictionary
        inpDict['core']['metal'] = metal # Shift the metal
        inpDict['core']['coreCN'] = cn # Shift the CN
        out_2D = build_complex_2D(inpDict) # Build in 2D
        mol2strings.append(out_2D['mol2string']) # Save the mol2string
        
df = pd.DataFrame({'metal':metals,'cn':coordination_numbers,'mol2string_2D':mol2strings}) # Create a dataframe

Now we can look at the full dataset we just generated:

In [ ]:
df

90 structures is quite a bit for a tutorial - Instead of doing all 90 structural generations Let's just do all the coordination number=5!

In [ ]:
gen_df = df[df.cn == 5].reset_index(drop=True) # Filter to only coordination 5
gen_df

Let's go! This will take a couple minutes

In [ ]:
lowest_energy_conformers = []
for i,row in gen_df.iterrows():
    trans_dict = copy.deepcopy(translate_dict)
    trans_dict['mol2string'] = row['mol2string_2D']
    out_3Ds = build_complex(trans_dict)
    key = list(out_3Ds.keys())[0]
    lowest_energy_conformers.append(out_3Ds[key]['mol2string'])
gen_df['mol2string_3D'] = lowest_energy_conformers # Save the output strings

Let's look at our generated dataframe:

In [ ]:
gen_df

And, we can directly visualize the generated 3D structures:

In [ ]:
view_structures(gen_df.mol2string_3D,labels=gen_df.metal.values)

### Looks pretty cool - xTB is  picking up some trends across the lanthanides.

# Conclusions

### In this tutorial we learned how to build in 2D and translate to 3D.  Specifically, we learned how to:

**(A)** How to generate in 2D.

**(B)** How to translate from 2D to 3D in Architector.

**(C)** How to perform 2D to 3D generation in an end-to-end workflow.